In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
data = pd.read_csv('winereviews.csv')
data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [3]:
reviews = []
for review in data['description']:
    reviews.append([review])
document = []
for review in data['description']:
    document.append(review)
document = ' '.join(document)
document = document.lower()
document = re.sub('[^a-zA-Z]', ' ', document )
re.sub(r'\s+', ' ', document)
document[:100]

'this tremendous      varietal wine hails from oakville and was aged over three years in oak  juicy r'

In [4]:
all_sentences = nltk.sent_tokenize(document)
all_words = [nltk.word_tokenize(sent) for sent in all_sentences]

In [5]:
from nltk.corpus import stopwords
for i in range(len(all_words)):
    all_words[i] = [w for w in all_words[i] if (w not in stopwords.words('english')) and w not in ['wine', 'drink', 'flavors', 'vineyard', 'finish', 'palate', 'drink', 'notes', 'very']]

In [6]:
from gensim.models import Word2Vec

In [7]:
word2vec = Word2Vec(all_words, min_count = 2, window = 5, size = 300)

In [8]:
vocabulary = word2vec.wv.vocab
word2vec.wv.most_similar('peach')

[('tannins', 0.7495031356811523),
 ('black', 0.7443265318870544),
 ('aromas', 0.739090621471405),
 ('offers', 0.738399088382721),
 ('ripe', 0.7307690382003784),
 ('juicy', 0.7255558967590332),
 ('cherry', 0.7175458669662476),
 ('white', 0.7131108641624451),
 ('blend', 0.7122121453285217),
 ('pear', 0.7053958773612976)]

In [9]:
query = ['dry']

In [10]:
### new
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from gensim.models.phrases import Phrases, Phraser

In [11]:
reviews = list(data['description'])
reviews = [str(r) for r in reviews]
reviews_full_doc = ' '.join(reviews)
sent_tokens = sent_tokenize(reviews_full_doc)

In [12]:
#remove stop words, lowercase, and punctuation
punc = '''!()-[]{};:.'"\, <>./?@#$%^&*_~'''

def process_text(text):
    words = word_tokenize(text)
    processed_sentence = []
    for word in words:
        w = str(word)
        lowercase = str.lower(w)
        stemmed = SnowballStemmer('english').stem(lowercase)
        remove_punc = stemmed
        for c in remove_punc:
            if c in punc:
                remove_punc.replace(c, "")
        processed_sentence.append(remove_punc)
    return(processed_sentence)
        
processed_sents = []
for sent in sent_tokens:
    processed = process_text(sent)
    processed_sents.append(processed)

In [13]:
# add bi grams/phrases
phrases = Phrases(processed_sents)
phrases = Phrases(phrases[processed_sents])
grams = Phraser(phrases)
phrase_sents = []
for sent in processed_sents:
    phrased = grams[sent]
    phrase_sents.append(phrased)
    
allwords = [item for sublist in phrase_sents for item in sublist]

In [14]:
allwords

['this',
 'tremend',
 '100_%',
 'variet',
 'wine',
 'hail_from',
 'oakvill',
 'and',
 'was',
 'age',
 'over',
 'three_year',
 'in',
 'oak',
 '.',
 'juici',
 'red-cherri',
 'fruit',
 'and',
 'a',
 'compel',
 'hint',
 'of',
 'caramel',
 'greet',
 'the',
 'palat',
 ',',
 'frame',
 'by',
 'eleg',
 ',',
 'fine',
 'tannin',
 'and',
 'a',
 'subtl',
 'minti',
 'tone',
 'in',
 'the',
 'background',
 '.',
 'balanc',
 'and',
 'reward',
 'from',
 'start',
 'to',
 'finish',
 ',',
 'it',
 'has',
 'year',
 'ahead',
 'of',
 'it',
 'to',
 'develop',
 'further',
 'nuanc',
 '.',
 'enjoy',
 '2022–2030',
 '.',
 'ripe',
 'aroma',
 'of',
 'fig',
 ',',
 'blackberri',
 'and',
 'cassi',
 'are',
 'soften',
 'and',
 'sweeten',
 'by',
 'a',
 'slather',
 'of',
 'oaki',
 'chocol',
 'and',
 'vanilla',
 '.',
 'this',
 'is',
 'full',
 ',',
 'layer',
 ',',
 'intens',
 'and',
 'cushion',
 'on',
 'the',
 'palat',
 ',',
 'with',
 'rich',
 'flavor',
 'of',
 'chocolati',
 'black',
 'fruit',
 'and',
 'bake',
 'spice',
 '.',
 

In [16]:
mapping = pd.read_csv('descriptor_mapping.csv').set_index('raw descriptor')

In [17]:
mapping.head()

,level_3,level_2,level_1
raw descriptor,,,
abras,abrasive,high_tannin,tannin
acacia,acacia,flowery,flower
acacia_flower,acacia,flowery,flower
aciddriven,acid_driven,high_acid,acid
aggress,aggressive,high_acid,acid


In [18]:
#map corpus words to level 3
def mapped_words(word):
    if word in list(mapping.index):
        mapped = mapping['level_3'][word]
        return mapped
    else:
        return word

processed_sents = []
for sent in phrase_sents:
    processed_sent = []
    for word in sent:
        mapped_word = mapped_words(word)
        processed_sent.append(str(mapped_word))
    processed_sents.append(processed_sent)

In [19]:
processed_sents

[['this',
  'tremend',
  '100_%',
  'variet',
  'wine',
  'hail_from',
  'oakvill',
  'and',
  'was',
  'age',
  'over',
  'three_year',
  'in',
  'oak',
  '.'],
 ['juicy',
  'red-cherri',
  'fruit',
  'and',
  'a',
  'compel',
  'hint',
  'of',
  'caramel',
  'greet',
  'the',
  'palat',
  ',',
  'frame',
  'by',
  'elegant',
  ',',
  'fine',
  'tannin',
  'and',
  'a',
  'subtl',
  'mint',
  'tone',
  'in',
  'the',
  'background',
  '.'],
 ['balanc',
  'and',
  'reward',
  'from',
  'start',
  'to',
  'finish',
  ',',
  'it',
  'has',
  'year',
  'ahead',
  'of',
  'it',
  'to',
  'develop',
  'further',
  'nuanc',
  '.'],
 ['enjoy', '2022–2030', '.'],
 ['ripe',
  'aroma',
  'of',
  'fig',
  ',',
  'blackberry',
  'and',
  'cassis',
  'are',
  'soften',
  'and',
  'sweet',
  'by',
  'a',
  'slather',
  'of',
  'oak',
  'chocolate',
  'and',
  'vanilla',
  '.'],
 ['this',
  'is',
  'full',
  ',',
  'layer',
  ',',
  'intens',
  'and',
  'cushion',
  'on',
  'the',
  'palat',
  ',',
 

In [20]:
#embeddings:

In [21]:
model = Word2Vec(processed_sents, size=300, min_count=1, iter=15)

In [22]:
print(model)

Word2Vec(vocab=40238, size=300, alpha=0.025)


In [39]:
print(model)

Word2Vec(vocab=40238, size=300, alpha=0.025)


In [23]:
model.wv.most_similar(positive='peach', topn=10)

[('pear', 0.6213881373405457),
 ('honeydew', 0.6180640459060669),
 ('lemon_curd', 0.6173605918884277),
 ('peach_nectar', 0.61366868019104),
 ('guava', 0.6105751395225525),
 ('nectarine', 0.6035938262939453),
 ('tangerine', 0.6025902032852173),
 ('grapefruit', 0.5972785949707031),
 ('mango', 0.5927752256393433),
 ('lemon_drop', 0.5896855592727661)]

In [24]:
#update review data to only show meaningful words
reviews = list(data['description'])
mapped_reviews = []
def mapping_wine_words(word):
    if word in list(mapping.index):
        mapped = mapping['level_3'][word]
        return mapped
for review in reviews:
    processed = process_text(review)
    phrased = grams[processed]
    mapped = [mapping_wine_words(word) for word in phrased]
    remove_none = [str(d) for d in mapped if d is not None]
    mapped_rev = ' '.join(remove_none)
    mapped_reviews.append(mapped_rev)

In [25]:
mapped_reviews

['oak juicy fruit caramel elegant mint',
 'ripe fig blackberry cassis sweet oak chocolate vanilla rich chocolate fruit baked spice toast heady',
 'complex white dark gold toasted_hazelnut pear compote orange_peel succulent residual_sugar',
 'oak fruit dense toast toast cigar_box blackberry cherry coffee graphite polished dark chocolate coffee',
 'dense wood rich concentrated',
 'depth dense dark ripe fruit oak blackberry coffee coffee toast oak spice smooth heady',
 'gritty sweet pastry prune blackberry coffee chocolate',
 'lush cedar almond vanilla tannin lush chocolate baked berry cassis hard tannin depth',
 'minerality chunky fruit citrus graphite concentrated',
 'bright crunchy cranberry pomegranate orange_peel silky succulent fleshy fruit lush length',
 'elegant complex white yellow spring_flower herb fruit cream juicy white peach ripe pear citrus white almond savory minerality',
 'supple coffee cherry vanilla savory',
 'green cola tobacco orange vanilla tea milk_chocolate ripe bl

In [26]:
## create embedding vectors for each review
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit(mapped_reviews)
dict_of_tfidf_weightings = dict(zip(X.get_feature_names(), X.idf_))

In [28]:
dict_of_tfidf_weightings

{'abrasive': 8.66189818067823,
 'acacia': 7.582243805754735,
 'aggressive': 6.330164597969767,
 'airy': 7.868332252371237,
 'allspice': 7.626260691171509,
 'almond': 4.746219892113709,
 'alpine_herbs': 9.523380676057391,
 'american_oak': 7.495232428765105,
 'angular': 6.7803924235938995,
 'anise': 5.193086028473862,
 'apple': 3.509584023742998,
 'apple_sauce': 8.381283275449542,
 'apricot': 4.429425997180188,
 'ash': 7.704222232641221,
 'asian_spice': 6.707971956634681,
 'asparagus': 7.168835844132634,
 'asphalt': 7.184785145540312,
 'assertive': 6.64231131082354,
 'astringent': 5.239334449743713,
 'austere': 5.734655886973738,
 'bacon': 5.614027899185123,
 'baked': 4.64234799442286,
 'baked_bread': 7.968751000118285,
 'balsamic': 6.1664835532918145,
 'banana': 5.85725820906607,
 'barbecue': 6.156084846070916,
 'bark': 7.849404242485718,
 'barnyard': 6.9231631797583955,
 'basil': 7.818632583818965,
 'bay_leaf': 8.493761258876232,
 'beef_jerky': 7.443939134377555,
 'beefy': 7.8006140783

In [29]:
review_vectors = []
for review in mapped_reviews:
    numwords = 0
    words = review.split(' ')
    tfidf_mapped_review = []
    for word in words:
        if word in dict_of_tfidf_weightings.keys():
            tfidf = dict_of_tfidf_weightings[word]
            vec = tfidf * model.wv.get_vector(word).reshape(1, 300)
            tfidf_mapped_review.append(vec)
            numwords = numwords + 1
    if len(tfidf_mapped_review) > 0:
        weight_rev_vec = sum(tfidf_mapped_review)/len(tfidf_mapped_review)
    else:
        weight_rev_vec = []
    review_vectors.append(weight_rev_vec)

In [54]:
print(model.wv)

In [30]:
query = ['asparagus']

In [31]:
import numpy as np
vectors_list = []
for vec in review_vectors:
    if len(vec) == 1:
        vectors_list.append(vec[0])
    else:
        vectors_list.append(np.zeros(300))

In [32]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=10, algorithm= 'brute', metric='cosine')
modelknn = knn.fit(vectors_list)

In [33]:
weightedqueryterms = []
for term in query:
    if term in dict_of_tfidf_weightings:
        tfidfweight = dict_of_tfidf_weightings[term]
        word_vector = tfidfweight * model.wv.get_vector(term).reshape(1,300)
        weightedqueryterms.append(word_vector)
query_vec = sum(weightedqueryterms)
distance, indice = modelknn.kneighbors(query_vec, n_neighbors = 5)

In [34]:
distance_list = distance[0].tolist()[1:]
indice_list = indice[0].tolist()[1:]

In [35]:
distance_list

[0.1520547414685124,
 0.1520547414685124,
 0.1520547414685124,
 0.15443217276513854]

In [36]:
indice_list

[65324, 121304, 116684, 66859]

In [37]:
for i in indice_list:
    print(i)
    review = mapped_reviews[i]
    location = data['province'][i]
    wine = data['variety'][i]
    print(location)
    print(wine)
    print(review)
    print()

65324
Other
Sauvignon Blanc
green_bean celery lime asparagus

121304
Other
Sauvignon Blanc
green_bean celery lime asparagus

116684
Other
Sauvignon Blanc
green_bean celery lime asparagus

66859
Loire Valley
Sauvignon Blanc
herbacious asparagus



In [38]:
data.iloc[130696]['description']

'This lightweight example of Marlborough Sauvignon Blanc features hints of green bean or asparagus as well as grapefruit flavors that fade quickly on the finish.'

In [ ]:
#new word embedding tfidf matrix of reviews
np.array(vectors_list).shape

In [ ]:
#cleaned and mapped reviews (with only wine words)
mapped_reviews

In [41]:
import pickle

In [44]:
with open('word2vecmodel.pickle', 'wb') as handle:
    pickle.dump(model.wv, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [45]:
with open('mapped_reviews.pickle', 'wb') as handle:
    pickle.dump(mapped_reviews, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [47]:
with open('tfidf_weight_dict.pickle', 'wb') as handle:
    pickle.dump(dict_of_tfidf_weightings, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [48]:
with open('review_vectors_list.pickle', 'wb') as handle:
    pickle.dump(dict_of_tfidf_weightings, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [49]:
with open('knnmodel.pickle', 'wb') as handle:
    pickle.dump(modelknn, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [50]:
print(modelknn)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)
